In [1]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg")  # Ou "Qt5Agg"
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Paramètres de la spirale
a = 0.3  # Facteur d'expansion de la spirale
b = 0.1  # Facteur de croissance du rayon
threshold = 5  # Distance seuil pour atteindre la cible

# Initialisation des positions
target_position = np.array([np.random.randint(0, 100), np.random.randint(0, 100)])
sensor_position = np.array([50,50])
target_history = [target_position.copy()]
tracking_position = np.copy(sensor_position)
tracking_history = [tracking_position.copy()]

fig, ax = plt.subplots()
ax.set_xlim(0, 100)
ax.set_ylim(0, 100)
target_line, = ax.plot([], [], 'r--', alpha=0.5, label="Trajectoire cible")
tracking_line, = ax.plot([], [], 'b--', alpha=0.5, label="Trajectoire suivi")
sensor_dot, = ax.plot([], [], 'go', markersize=8, label="Capteur")
target_dot, = ax.plot([], [], 'ro', markersize=8, label="Cible")
tracking_dot, = ax.plot([], [], 'bo', markersize=8, label="Tracking")
ax.legend()
ax.set_title("Simulation Pointing, Acquisition, and Tracking (PAT)")


Text(0.5, 1.0, 'Simulation Pointing, Acquisition, and Tracking (PAT)')

In [2]:
def update_target():
    """Simule le mouvement de la cible et met à jour l'historique."""
    global target_position
    new_position = target_position + np.random.randint(-1, 2, size=2)
    target_position = np.clip(new_position, 0, 100)  # Garde dans les limites

    # Limiter la taille de l'historique pour éviter une surcharge mémoire
    if len(target_history) > 50:
        target_history.pop(0)
    target_history.append(target_position.copy())

In [3]:

def track_unfound_target(frame):
    """Simule le suivi de la cible par ajustement progressif."""
    global tracking_position
    theta = a * frame  # Angle qui tourne
    r = b * frame      # Rayon qui grandit
    x = 50 + r * np.cos(theta)
    y = 50 + r * np.sin(theta)
    new_position = np.array([x,y])
    tracking_position = np.clip(new_position, 0, 100)
    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())


In [4]:
def track_found_target():
    """Simule le suivi de la cible par ajustement progressif."""
    global tracking_position
    direction = target_position - tracking_position
    step = np.clip(direction, -2, 2)  # Pas de mouvement limité pour un suivi progressif
    tracking_position += step
    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())  # Sauvegarde historique

In [5]:
def update(frame):
    global tracking_position, target_position
    distance = np.sqrt((tracking_position[0] - target_position[0]) ** 2 + (tracking_position[1] - target_position[1]) ** 2)
    if distance>=threshold:
        update_target()
        track_unfound_target(frame)
    else :
        update_target()
        track_found_target()

    # Mise à jour des données des objets graphiques
    target_x, target_y = zip(*target_history)
    tracking_x, tracking_y = zip(*tracking_history)

    target_line.set_data(target_x, target_y)
    tracking_line.set_data(tracking_x, tracking_y)
    target_dot.set_data(target_position[0], target_position[1])
    sensor_dot.set_data(sensor_position[0], sensor_position[1])
    tracking_dot.set_data(tracking_position[0], tracking_position[1])

    return target_line, target_dot, sensor_dot, tracking_line, tracking_dot,

In [6]:
ani = animation.FuncAnimation(fig, update, frames=1000, interval=100, blit=False)
plt.show()